In [1]:
import math
import torch
from torch import empty , cat , arange
from torch . nn . functional import fold , unfold

#to delete
from torch import nn
from others.modules import *


In [2]:
# Extract images

noisy_imgs_1 , noisy_imgs_2 = torch.load('data/train_data.pkl')
noisy_imgs , clean_imgs = torch.load ('data/val_data.pkl')

noisy_imgs = noisy_imgs/255
clean_imgs = clean_imgs/255

# select a preset of images:

imgs_1 = noisy_imgs_1[:10000]/255
imgs_2 = noisy_imgs_2[:10000]/255
print(imgs_1.shape)

torch.Size([10000, 3, 32, 32])


In [19]:
class Model(object):
    def __init__(self):
        self.sequence = Sequential(Convolution(3, 20, kernel_size = 4, stride = 2),
                                   ReLU(),
                                   Convolution(20,20, kernel_size = 3, stride = 2),
                                   ReLU(),
                                   Upsample(2),
                                   Convolution(20,20, kernel_size = 3, stride = 1),
                                   ReLU(),
                                   Upsample(3),
                                   Convolution(20,3, kernel_size = 5, stride = 1),
                                   Sigmoid())
                                   

        self.criterion = MSE()
        self.optimizer = SGD(self.sequence, 1e-2)
        
        self.nb_samples = 10000
        self.epochs = 10
        print('Model Initialisation')

    def train(self,train_input,train_target):
    
        for epoch in range(self.epochs):

            if not epoch % 5: 
                    print('epoch',epoch)

            for n in range(self.nb_samples):

                if not n % 10000: 
                    print('sample: ', n)
                    print('weight',self.sequence.layers[0].weight[0,0,0])
                    print('grad',self.sequence.layers[0].grad[0,0,0])
                #print('img', train_input[n:n+1].shape)

                predict_img = self.sequence.forward(train_input[n:n+1])
                #print('optimizer zero grad')
                self.optimizer.zero_grad()
                #print('prediction', predict_img.shape)
                loss = self.criterion.forward(predict_img, train_target[n:n+1])
                #print('loss backward', self.criterion.backward())
                backward_img = self.sequence.backward(self.criterion.backward())
                #if not n % 100:
                    #print('loss', loss)
                    #print('grad conv0',self.sequence.layers[0].grad[0,0,0])
                    #print('grad sigmoid',self.sequence.layers[8].grad[0,0,0,0])
                    #print('grad upsample',self.sequence.layers[7].grad[0,0,0,0])
                    #print('grad relu',self.sequence.layers[6].grad[0,0,0,0])
                    #print('*********************************************************************************')
                self.optimizer.step()
        

        return backward_img
    
    def predict(self,test_input):
        prediction = torch.empty(test_input.shape)
            
        for n in range(test_input.shape[0]):
            prediction[n:n+1] = self.sequence.forward(test_input[n:n+1])

        return prediction
        

In [20]:
#img = torch.randn(1, 3, 32, 32)
#img_error = torch.randn(1, 3, 32, 32)

model = Model()

output = model.train(imgs_1, imgs_2)



print('out img', output.shape)



Model Initialisation
epoch 0
sample:  0
weight tensor([ 0.5270,  0.7905,  0.5566, -0.0150])
grad tensor([0., 0., 0., 0.])
sample:  100
weight tensor([ 0.4690,  0.7785,  0.5475, -0.0872])
grad tensor([0., 0., 0., 0.])
sample:  200
weight tensor([ 0.4732,  0.7767,  0.5493, -0.0942])
grad tensor([0., 0., 0., 0.])
sample:  300
weight tensor([ 0.4736,  0.7770,  0.5495, -0.0941])
grad tensor([0., 0., 0., 0.])
sample:  400
weight tensor([ 0.4678,  0.7705,  0.5430, -0.0996])
grad tensor([0., 0., 0., 0.])
sample:  500
weight tensor([ 0.4786,  0.7818,  0.5458, -0.0929])
grad tensor([0., 0., 0., 0.])
sample:  600
weight tensor([ 0.5041,  0.8267,  0.5824, -0.0450])
grad tensor([0., 0., 0., 0.])
sample:  700
weight tensor([ 0.4745,  0.7981,  0.5523, -0.0667])
grad tensor([0., 0., 0., 0.])
sample:  800
weight tensor([ 0.4776,  0.7960,  0.5553, -0.0625])
grad tensor([0., 0., 0., 0.])
sample:  900
weight tensor([0.5435, 0.8468, 0.5925, 0.0157])
grad tensor([ 0.8798, -0.0661,  1.9744, -0.8556])
sample:

sample:  8900
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9000
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9100
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9200
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9300
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9400
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9500
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9600
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9700
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9800
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0., 0., 0.])
sample:  9900
weight tensor([0.7474, 1.0710, 0.8139, 0.2613])
grad tensor([0., 0

sample:  7800
weight tensor([0.5458, 0.8633, 0.7469, 0.0716])
grad tensor([0., 0., 0., 0.])
sample:  7900
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0., 0., 0.])
sample:  8000
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0., 0., 0.])
sample:  8100
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0., 0., 0.])
sample:  8200
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0., 0., 0.])
sample:  8300
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0., 0., 0.])
sample:  8400
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0., 0., 0.])
sample:  8500
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0., 0., 0.])
sample:  8600
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0., 0., 0.])
sample:  8700
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0., 0., 0.])
sample:  8800
weight tensor([0.5457, 0.8633, 0.7467, 0.0714])
grad tensor([0., 0

sample:  6800
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6900
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  7000
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  7100
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  7200
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  7300
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  7400
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  7500
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  7600
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  7700
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  7800
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0

sample:  5800
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  5900
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6000
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6100
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6200
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6300
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6400
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6500
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6600
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6700
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0., 0., 0.])
sample:  6800
weight tensor([0.5495, 0.8666, 0.7490, 0.0706])
grad tensor([0., 0

...

sample:  2700
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  2800
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  2900
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  3000
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  3100
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  3200
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  3300
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  3400
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  3500
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  3600
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  3700
weight tensor([-0.9668, -0

sample:  1300
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  1400
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  1500
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  1600
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  1700
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  1800
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  1900
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  2000
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  2100
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  2200
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  2300
weight tensor([-0.9668, -0

sample:  9900
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  0
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  100
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  200
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  300
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  400
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  500
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  600
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  700
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  800
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  900
weight tensor([-0.9668, -0.3117,  0.14

sample:  8500
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  8600
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  8700
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  8800
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  8900
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  9000
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  9100
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  9200
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  9300
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  9400
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  9500
weight tensor([-0.9668, -0

sample:  7100
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  7200
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  7300
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  7400
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  7500
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  7600
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  7700
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  7800
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  7900
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  8000
weight tensor([-0.9668, -0.3117,  0.1411, -0.1111])
grad tensor([0., 0., 0., 0.])
sample:  8100
weight tensor([-0.9668, -0

In [11]:
def psnr ( denoised , ground_truth ) :
        # Peak Signal to Noise Ratio : denoised and ground_truth have range [0 , 1]
        mse = torch.mean (( denoised - ground_truth ) ** 2)
        return -10 * torch.log10 ( mse + 10** -8)

In [12]:
print(noisy_imgs.shape[0])

prediction = model.predict(noisy_imgs)


print(prediction.shape)

1000
torch.Size([1000, 3, 32, 32])


In [13]:
img_denoise_psnr = psnr(prediction ,clean_imgs )

In [14]:
print("psnr", img_denoise_psnr)

psnr tensor(5.9272)
